## 1. Create the compute instance

In [ ]:
# import required libraries
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import WorkspaceConnection
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

from azure.ai.ml.entities import ComputeInstance
from azure.ai.ml.entities import CustomApplications, ImageSettings, EndpointsSettings, VolumeSettings 

ml_client = MLClient.from_config(credential=credential)

In [ ]:
# create custom application from default promptflow image
image = ImageSettings(reference='mcr.microsoft.com/azureml/promptflow/promptflow-runtime-stable:latest') 
endpoints = [EndpointsSettings(published=8081, target=8080)]
app = CustomApplications(name='promptflow-runtime',endpoints=endpoints,bind_mounts=[],image=image,environment_variables={}) 

In [ ]:
# create compute instance for prompt flow
ci_basic_name = "contoso-chat-ci"
ci_basic = ComputeInstance(name=ci_basic_name, size="STANDARD_DS12_V2",custom_applications=[app])

# check if compute instance is already created
try:
    ci_instance = ml_client.compute.get(name=ci_basic_name)
    print("Compute instance already exists and is current state is:", ci_instance.state)
except:
    ci_instance = ml_client.begin_create_or_update(ci_basic)
    print("Creating compute instnace", ci_instance)
    # Check state of compute instance
    ci_instance = ml_client.compute.get(name=ci_basic_name)
    print(ci_instance.state)

In [ ]:
# List compute instances
for ci in ml_client.compute.list():
    print(ci.name)

## 2. Create the prompt flow runtime

Once the compute instance is created. [Follow these instructions to create a runtime in AI Studio.](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-manage-runtime)

## 3. Run the prompt flow on the runtime to install the dependencies

In [ ]:
config_path = "../config.json"
from promptflow.azure import PFClient
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)


In [ ]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "."
run_name="update_runtime_packages"
data = "../data/emptydata.jsonl"

In [ ]:
try:
    runtime_flow = pf_azure_client.flows.create_or_update(
        flow=".",
        runtime=runtime,
        display_name=run_name)
    print("Creating prompt flow", runtime_flow)
except:
    runtime_flows = pf_azure_client.flows.list()
    # loop thru flows and get the one based on the run_name
    for runtime_flow in runtime_flows:
        if runtime_flow.display_name == run_name:
            runtime_flow = runtime_flow
            print("Found runtime flow", runtime_flow)
            break

In [ ]:
runtime_flow_run = pf_azure_client.run(
    flow=flow,
    data=data,
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(runtime_flow_run)

In [ ]:
pf_azure_client.stream(runtime_flow_run)

In [ ]:
details = pf_azure_client.get_details(runtime_flow_run)

In [ ]:
details["outputs.version"][0]